# Przygotowanie

In [171]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

import tensorflow
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Bidirectional, SpatialDropout1D
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [172]:
from const import W2V_MODEL_PATH, PREPROCESSED_TEST_PATH, PREPROCESSED_TRAIN_PATH

Wczytywanie danych

In [173]:
df_train = pd.read_pickle(PREPROCESSED_TRAIN_PATH)
df_test = pd.read_pickle(PREPROCESSED_TEST_PATH)

Wczytywanie w2v

In [174]:
w2v_model = Word2Vec.load(W2V_MODEL_PATH)

# Odzielanie danych od labeli

In [175]:
Y_train = df_train.target.to_numpy().reshape(-1,1)
Y_test = df_test.target.to_numpy().reshape(-1,1)

Jeszcze musimy zamienić lity tekst na tokeny. W tym celu skorzystamy z klasy __Tokenizera__ z `kerasa`.

In [176]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

X_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen = 50)
X_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen = 50)

In [177]:
print("X_train", X_train.shape)
print("Y_train", Y_train.shape)
print()
print("X_test", X_test.shape)
print("Y_test", Y_test.shape)

X_train (1280000, 50)
Y_train (1280000, 1)

X_test (320000, 50)
Y_test (320000, 1)


In [178]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   73,
        334, 4912,  123,   75,  147, 3268], dtype=int32)

In [179]:
Y_train[0]

array([1])

In [180]:
def create_embedding_layer():
    vocab_size = len(tokenizer.word_index) + 1
    embedding_matrix = np.zeros((vocab_size, 300))
    for word, i in tokenizer.word_index.items():
        if word in w2v_model.wv:
            embedding_matrix[i] = w2v_model.wv[word]
    return Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = 50, trainable = False)

# Model

In [181]:
models = []

In [182]:
print("Training on GPU...") if tensorflow.config.list_physical_devices('GPU') else print("Training on CPU...")

Training on CPU...


### LSTM_V1
- własny embedding
- LSTM 100

In [183]:
model = Sequential(name="LSTM_V1")
model.add(create_embedding_layer())
model.add(Dropout(0.5))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

### LSTM_V2
- własny embedding
- convy
- dwukierunkowy LSTM 64
- największy dens

In [184]:
model = Sequential(name="LSTM_V2")
model.add(create_embedding_layer())
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

### LSTM_V3
- embedding uczony na bierząco
- LSTM 50
- najmniejsza sieć

In [185]:
model = Sequential(name="LSTM_V3")
model.add(Embedding((len(tokenizer.word_index) + 1), 32, input_length=50))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))

models.append(model)

In [ ]:
for model in models:
    model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    
    callbacks = [ 
        ReduceLROnPlateau(monitor = 'val_loss', patience = 5),
        EarlyStopping(monitor = 'val_acc', patience = 5)
    ]
    
    history = model.fit(X_train, Y_train,
                        batch_size = 1024,
                        epochs = 1,
                        validation_split = 0.1,
                        verbose = 1,
                        callbacks=callbacks)
    
    np.save("{}_history.npy".format(model.name), history.history)
    model.save("{}.model".format(model.name))
    

Train on 1152000 samples, validate on 128000 samples
Epoch 1/1
  24576/1152000 [..............................] - ETA: 9:27 - loss: 0.6590 - accuracy: 0.6311